<h2> Introduction </h2<

In [ ]:
# Importing python modules
import pandas as pd
import warnings
import mlflow
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score
warnings.filterwarnings(action= "ignore", message= "Ignoring Warning")

In [ ]:
loan_df = pd.read_csv('./dataset/loan_data_set.csv')  # using pandas read_csv function to load the loan data.

In [ ]:
loan_df.head(20)  # returns first 20 samples of the loan data

In [ ]:
loan_df.describe()  # describes statistical information on integer columns of the data.

In [ ]:
loan_df.info()  # shows information on the loan data like columns with null data and datatype of each column.

In [ ]:
loan_df.isnull().sum()  # counts number null values present in each columns.

In [ ]:
# creating a list of columns with null values
col_null = []
column = list(loan_df.columns)
print(column)
for col in column:
    if loan_df[col].isna().sum() > 0:
        col_null.append(col)
print(col_null)

In [ ]:
# creating a list of the modes for each of the columns with missing values
mode = []
for miss in col_null:
    mode.append(loan_df[miss].mode())
print(mode)

In [ ]:
# picks the modes in each values inside the list.
real_mode = []
for modes in mode:
    real_mode.append(modes[0])
print(real_mode)

In [ ]:
# creates a dictionary where the keys are the missing columns and the values show the mode of each columns
val = {}
for miss in col_null:
    for m in real_mode:
        val[miss] = m
        real_mode.remove(m)
        break

In [ ]:
print(val)

In [ ]:
# fills the missing values in each column
for i,k in val.items():
    loan_df[i].fillna(k, inplace= True)

In [ ]:
loan_df.info()  # checking number of missing values present.

In [ ]:
loan_df.isnull().sum()

In [ ]:
#  identifying columns with object datatype
s = (loan_df.dtypes == 'object')

In [ ]:
# creating list of objects with object datatypes.
obj_col = list(s[s].index)
obj_col.remove("Loan_ID")  # removing the id column

In [ ]:
obj_col

In [ ]:
ordinate = OrdinalEncoder()  # the ordinate object takes the OrdinalEncoder class as it's values.

In [ ]:
loan_data = loan_df.copy()  # making a copy of the loan_df to be stored with loan_data

In [ ]:
# transforming the columns with object datatypes.
loan_data[obj_col] = ordinate.fit_transform(loan_data[obj_col])

In [ ]:
loan_data.head()  # outputs top 5 rows of the dataset.

In [ ]:
X = loan_data.drop(["Loan_Status", "Loan_ID"], axis= 1)  # variable X after dropping off the Loan_Status and Loan_ID
y = loan_data.Loan_Status  # variable y takes the Loan Status column.

In [ ]:
scale = StandardScaler()  # creating a variable named scale saving the StandardScaler class
rob = RobustScaler()  # creating a variable named rob saving the RobustScaler class.

In [ ]:
# transforming the variable X using the scale and robust class
scaled_X = scale.fit_transform(X)
rob_X = rob.fit_transform(X, y)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(rob_X, y, random_state=1234, test_size= 0.2)

In [ ]:
# creating variables to store algorithm classes
lr = LogisticRegression(random_state= 1234, n_jobs= 1)
svc = SVC(random_state= 1234, C =1, kernel= 'poly', degree= 0, max_iter= 1)
rf = RandomForestClassifier(n_estimators= 100, n_jobs= 1, max_leaf_nodes= 6)

In [ ]:
voting_clf = VotingClassifier(
    estimators= [('lr', lr), ('rf', rf), ('svc', svc)],
    voting = "hard",
)

In [ ]:
voting_clf.fit(train_X, train_y)

In [ ]:
for clf in (lr, rf, svc, voting_clf):
    clf.fit(train_X, train_y)
    y_pred = clf.predict(test_X)
    print(clf.__class__.__name__, accuracy_score(y_pred, test_y), f1_score(y_pred, test_y))

In [ ]:
new_file = {}
details = ['Male', 'No', 1, 'Not Graduate', 'No', int(2000), int(0), int(55), int(180), int(0), 'Rural']
column.remove('Loan_ID')
column.remove('Loan_Status')
for col in column :
    for det in details:
        new_file[col] = det
        details.remove(det)
        break

In [ ]:
import numpy as np
detail = ['Male', 'No', 1, 'Not Graduate', 'No', int(2000), int(0), int(55), int(180), int(0), 'Rural']
np_det = np.array([detail])

In [ ]:
print(np_det.shape)
new_data = pd.DataFrame(new_file, columns= column, index = range(1))

In [ ]:
new_fil = np.array(new_file)

In [ ]:
new_file

In [ ]:
new_data.info()

In [ ]:
new_data

In [ ]:
y = (new_data.dtypes == 'object')
obj = list(y[y].index)
new_data[obj] = ordinate.fit_transform(new_data[obj])

In [ ]:
new_data

In [ ]:
new_pred = voting_clf.predict(new_data)

In [ ]:
new_pred